In [ ]:
import seaborn as sn
from keras.models import Sequential
from keras.layers import Dense,SimpleRNN, Conv1D, Flatten
from sklearn.metrics import mean_squared_error

## 4. CNN *vs* RNN Prediciendo el Ozono Atmosférico

**a)**

In [ ]:
data = pd.read_csv('./ozone_data.csv') 
data_train, data_val, data_test = np.split(data.sample(frac=1), [int(.6*len(data)), int(.8*len(data))])

In [ ]:
X_train = data_train.values[:,1:]
X_test = data_test.values[:,1:]
X_val = data_val.values[:,1:]

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
scaler.fit(X_val)
X_val_scaled = scaler.transform(X_val)
scaler.fit(X_test)
X_test_scaled = scaler.transform(X_test)

In [ ]:
plt.subplots(figsize = (12,12))
sn.heatmap(data.corr())

**b)**

In [ ]:
def create_dataset(dataset,lag=1):
    dataX = []
    dataY = []
    dataset = dataset.flatten()
    for i in range(lag+1, len(dataset)):
        dataX.append(dataset[i-(lag+1):i-1])
        dataY.append(dataset[i])
    return np.array(dataX),np.array(dataY)

In [ ]:
lag = 3
trainX, trainY = create_dataset(X_train_scaled, lag)
testX, testY = create_dataset(X_test_scaled, lag)
valX, valY = create_dataset(X_val_scaled, lag)

**c)**

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
#valX = np.reshape(valX, (valX.shape[0], 1, valX.shape[1]))

In [ ]:
model_rnn = Sequential()
model_rnn.add(SimpleRNN(units=4, input_shape=(1,lag), activation='tanh',return_sequences=False,go_backwards=False))
model_rnn.add(Dense(1,activation='linear'))
model_rnn.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model_rnn.summary()
rnn_weights = model_rnn.get_weights()
model_rnn.fit(trainX, trainY, epochs=25, batch_size=1, verbose=1)

In [ ]:
model_cnn = Sequential()
model_cnn.add(Conv1D(4, 1,input_shape=(1,lag), activation='relu',padding='valid')) 
model_cnn.add(Flatten())
model_cnn.add(Dense(1,activation='linear'))
model_cnn.compile(loss='mean_squared_error', optimizer='adam')
model_cnn.summary()
cnn_weights = model_cnn.get_weights()
model_cnn.fit(trainX, trainY, epochs=25, batch_size=1, verbose=1)

**d)**

In [ ]:
## RNN
trainP_RNN = model_rnn.predict(trainX,batch_size=1)
trainPredict_RNN = scaler.inverse_transform(trainP_RNN)
trainYre_RNN = trainY.reshape(-1,1)
trainYre_RNN = scaler.inverse_transform([trainYre_RNN])

testP_RNN = model_rnn.predict(testX,batch_size=1)
testPredict_RNN = scaler.inverse_transform(testP_RNN)
testYre_RNN = testY.reshape(-1,1)
testYre_RNN = scaler.inverse_transform([testYre_RNN])

# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainYre_RNN, trainPredict_RNN))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testYre_RNN, testPredict_RNN))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# CNN
trainP_CNN = model_cnn.predict(trainX,batch_size=1)
trainPredict_CNN = scaler.inverse_transform(trainP_CNN)
trainYre_CNN = trainY.reshape(-1,1)
trainYre_CNN = scaler.inverse_transform([trainYre_CNN])

testP_CNN = model_cnn.predict(testX,batch_size=1)
testPredict_CNN = scaler.inverse_transform(testP_CNN)
testYre_CNN = testY.reshape(-1,1)
testYre_CNN = scaler.inverse_transform([testYre_CNN])

# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainYre_CNN, trainPredict_CNN))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testYre_CNN, testPredict_CNN))
print('Test Score: %.2f RMSE' % (testScore))

**e)**

**f)**

**g)**

**h)**

**k)**

**i)**